In [1]:
%%capture
!pip install keras_tuner
!pip install tensorflow

In [2]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import (
    load_model,
    clone_model
)
import tensorflow as tf

In [3]:
import numpy as np
import os
import sys
from google.colab import userdata
s = userdata.get('ghkey')
with open('./github_token', 'wb') as f:
  f.write(bytes(s, 'utf'))
!rm -rf ./207_final
!git clone https://oauth2:$(cat ./github_token)@github.com/numbersrcool/207_final.git

Cloning into '207_final'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 287 (delta 60), reused 102 (delta 56), pack-reused 179 (from 1)
Receiving objects: 100% (287/287), 107.38 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [4]:
%%capture
b = userdata.get('bucket')
with open('./.bucket', 'wb') as f:
  f.write(bytes(b, 'utf'))

!pip install google-cloud-storage

from google.colab import auth
auth.authenticate_user()
!rm -rf data/
!gsutil cp gs://$(cat /content/.bucket)/207_final.zip  /content/
!unzip 207_final.zip

In [5]:

%cd 207_final

/content/207_final


In [6]:
%env MIDS_207_DATA_PATH=../data

from datasets import (
    training,
    validation,
    TRAIN_BATCH_SIZE,
    TRAIN_NUM_BATCHES,
)


env: MIDS_207_DATA_PATH=../data
loading data from ../data...
splitting data into train, val, test...


In [ ]:
from models.cnn_B import tuner

In [ ]:
tuner.search(training.data, epochs=10, validation_data=validation.data)

Trial 10 Complete [00h 01m 07s]
val_accuracy: 0.996874988079071

Best val_accuracy So Far: 0.996874988079071
Total elapsed time: 00h 07m 07s


In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
tf.keras.backend.clear_session()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

history = model.fit(training.data,
    epochs=TRAIN_NUM_BATCHES,
    batch_size=TRAIN_BATCH_SIZE,
    validation_data=validation.data,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.7874 - loss: 1.2961 - val_accuracy: 0.9953 - val_loss: 0.0239 - learning_rate: 0.0010
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.9821 - loss: 0.0648 - val_accuracy: 0.9953 - val_loss: 0.0196 - learning_rate: 0.0010
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.9915 - loss: 0.0372 - val_accuracy: 0.9969 - val_loss: 0.0173 - learning_rate: 0.0010
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.9945 - loss: 0.0223 - val_accuracy: 0.9859 - val_loss: 0.0557 - learning_rate: 0.0010
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 90ms/step - accuracy: 0.9949 - loss: 0.0203 - val_accuracy: 0.9984 - val_loss: 0.0058 - learning_rate: 0.0010
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 89ms/step - accuracy: 0.9974 - loss: 0.0164 - val_accuracy: 0.9984 - val_loss: 0.0045 - learning_rate: 0.0010
Epoch 7/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.9977 -

In [ ]:
model.save('cnn_B.keras')

In [7]:
# model = load_model('./models/cnn_B.keras')
baseline_model = clone_model(model)

In [13]:
# # Training the baseline model

tf.keras.backend.clear_session()

# Unclear if these are mutated from the last call, so we'll just recreate.
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Compile from the same config
baseline_model.compile_from_config(model.get_compile_config())

history = baseline_model.fit(training.baseline_data,
    epochs=TRAIN_NUM_BATCHES,
    batch_size=TRAIN_BATCH_SIZE,
    validation_data=validation.baseline_data,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 57ms/step - accuracy: 0.6719 - loss: 2.1380 - val_accuracy: 0.9984 - val_loss: 0.0145 - learning_rate: 5.0000e-04
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9901 - loss: 0.1020 - val_accuracy: 1.0000 - val_loss: 2.6371e-04 - learning_rate: 5.0000e-04
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 1.0000 - loss: 9.1860e-05 - val_accuracy: 1.0000 - val_loss: 1.0091e-05 - learning_rate: 5.0000e-04
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 1.0000 - loss: 3.2771e-06 - val_accuracy: 1.0000 - val_loss: 4.0605e-07 - learning_rate: 5.0000e-04
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 1.0000 - loss: 1.7748e-06 - val_accuracy: 1.0000 - val_loss: 2.3340e-06 - learning_rate: 5.0000e-04
Epoch 6/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 1.0000 - loss: 3.7856e-07 - val_accuracy: 1.0000 - val_loss: 3.2391e-07 - learning_rate: 2.5000e-04
Epoch 7/100
100

In [14]:
baseline_model.save('/content/cnn_B_baseline.keras')